# Embeddings
본 노트북에서는, [자습서: Azure OpenAI Service 임베딩 및 문서 검색 살펴보기](https://learn.microsoft.com/en-us/azure/ai-services/openai/tutorials/embeddings?tabs=python-new%2Ccommand-line&pivots=programming-language-python)를 이용하여 Azure OpenAI 임베딩 API를 사용하여 문서 검색을 수행하는 과정을 실습합니다. text-embedding-ada-002 (Version 2) 모델을 이용하여 Wiki 데이터를 벡터 검색을 할 수 있도록 변환합니다.

Azure OpenAI 리소스가 사전에 생성되어 있어야 합니다.

### 소요 시간

이 노트북을 실행하는데는 15분 정도 소요됩니다.

In [ ]:
# Install the packages\
%pip install requests openai~=1.10

#### 라이브러리 가져오기

In [69]:
import os
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
import tiktoken
from openai import AzureOpenAI
from dotenv import load_dotenv

#### CSV 파일을 읽고, Pandas DataFrame을 만듭니다.

In [ ]:
df=pd.read_csv(os.path.join(os.getcwd(),'./data/wiki_data.csv')) # This assumes that you have placed the bill_sum_data.csv in the same directory you are running Jupyter Notebooks
df

#### ID, URL, TITLE, TEXT를 포함하는 df_wiki라는 새 DataFrame을 만듭니다.

In [ ]:
df_wiki = df[['id', 'url', 'title', 'text']]
df_wiki

#### 불필요한 공백을 제거하고 문장 부호를 정리하여 토큰화를 위한 데이터를 준비하여 간단한 데이터 정리를 수행합니다.

In [52]:
pd.options.mode.chained_assignment = None #https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#evaluation-order-matters

# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df_wiki['text']= df_wiki["text"].apply(lambda x : normalize_text(x))

#### 토큰 제한에 비해 너무 긴 Text를 제거합니다.

In [ ]:
tokenizer = tiktoken.get_encoding("cl100k_base")
df_wiki['n_tokens'] = df_wiki["text"].apply(lambda x: len(tokenizer.encode(x)))
df_wiki = df_wiki[df_wiki.n_tokens<8192]
len(df_wiki)

#### df_wiki를 검토합니다.

In [ ]:
df_wiki

#### n_tokens 열과 텍스트가 궁극적으로 토큰화 되는 방식을 조금 더 이해하려면 다음 코드를 실행하는 것이 도움이 될 수 있습니다.

In [ ]:
sample_encode = tokenizer.encode(df_wiki.text[0]) 
decode = tokenizer.decode_tokens_bytes(sample_encode)
decode

In [ ]:
len(decode)

#### Azure OpenAI의 Embbeding 모델을 사용하여 Embbeding을 생성합니다. df_wiki['content_vector']에 "Text" 열에서 "text-embedding-ada-002(Version 2)" 모델로 embedding을 생성합니다.

In [60]:
load_dotenv()
azure_oai_endpoint = os.getenv("AZURE_OAI_ENDPOINT")
azure_oai_key = os.getenv("AZURE_OAI_KEY")
azure_oai_deployment = os.getenv("AZURE_OAI_DEPLOYMENT")

client = AzureOpenAI(
  azure_endpoint="https://aoai-lab-jl.openai.azure.com/",
  api_key="3722670f4c384a2fa559f748426468f8",
  api_version = "2024-02-01",)

def generate_embeddings(text, model="text-embedding-ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

df_wiki['content_vector'] = df_wiki["text"].apply(lambda x : generate_embeddings (x, model = 'text-embedding-ada-002'))
# model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model

#### df_wiki의 "'id', 'url', 'title', 'text', 'n_tokens', 'content_vector'" 열을 "wiki_data_embeddings.csv"라는 CSV 파일에 저장합니다.

In [67]:
df_wiki = df_wiki[['id', 'url', 'title', 'text', 'n_tokens', 'content_vector']]

df_wiki

df_wiki.to_csv('./data/wiki_data_embeddings.csv', index=False)

#### 아래의 Search 코드 블록을 실행합니다. "4월과 8월에 대한 정보를 알려줘"라는 검색 쿼리를 포함합니다. Cosine 유사성으로 4월과 8월에 대한 정보에 가장 가깝게 순위가 매겨진 Text를 찾습니다.

In [70]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_embedding(text, model="text-embedding-ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

def search_docs(df, user_query, top_n=4, to_print=True):
    embedding = get_embedding(
        user_query,
        model="text-embedding-ada-002" # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )
    df["similarities"] = df.content_vector.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res


res = search_docs(df_wiki, "Can I get the information abour April and August?", top_n=4)

,id,url,title,text,n_tokens,content_vector,similarities
1,2,https://simple.wikipedia.org/wiki/August,August,August (Aug.) is the eighth month of the year ...,2179,"[0.0011972341453656554, 0.0028177648782730103,...",0.805780
0,1,https://simple.wikipedia.org/wiki/April,April,April is the fourth month of the year in the J...,3902,"[-0.011200855486094952, -0.016985775902867317,...",0.798313
24,48,https://simple.wikipedia.org/wiki/Astronomy,Astronomy,Astronomy (from the Greek astron (ἄστρον) mean...,2564,"[0.020063724368810654, 0.011303506791591644, 0...",0.710493
28,52,https://simple.wikipedia.org/wiki/Afghanistan,Afghanistan,"Afghanistan, officially the Islamic Emirate of...",3992,"[-0.00850854441523552, -0.02313825488090515, -...",0.708763


#### 이 방식을 사용하면 자료의 문서 전체에 Embedding을 검색 메커니즘으로 사용할 수 있습니다. 사용자는 상위 검색 결과를 가져와 다운스트림 작업에 사용할 수 있습니다.